Complete the code in the blocks below. Do not alter blocks that indicate that they should not be changed.

Look for blocks that say "FILL IN THIS SECTION"

FILL IN THIS SECTION

Author(s): Emilio Cardenas, Johan Olvera, Theodore Weller


In this homework we will build and test a Hidden Markov Model for text generation.

In [22]:
# Import statements here if needed
from urllib.request import urlopen
from urllib.parse import unquote
import random
import re # imported regex

In [33]:
# DO NOT CHANGE
# Fetch a body of text from a web-hosted file
url = "https://github.com/gentry-atkinson/small_text_repo/raw/main/alice29.txt"
page = urlopen(url)
page = page.read()
page = page.decode('utf8')
alice_text = ''
for ch in page:
  alice_text += ch

# Remove some blank lines
alice_text = '\n'.join([line.strip().upper() for line in alice_text.split('\n') if len(line) > 0 and line not in ['', ' ', '\n', ' \n']])

def trim_word(word: str) -> str:
    if not word:
        return None

    # Determine the start and end indices ignoring specified punctuations
    a, b = 0, len(word) - 1
    while a <= b and word[a] in ' .!?\';:"(`-':
        a += 1
    while b >= a and word[b] in ' .!?\';:"`)`,.-':
        b -= 1

    # Return None if there are no valid characters (all punctuation)
    if a > b:
        return None
    return word[a:b+1]

#Check text
print(alice_text[0:50])






ALICE'S ADVENTURES IN WONDERLAND

LEWIS CARROL


In [26]:
# FILL IN THIS SECTION

word_count = {}
total_count = 0


# Split the text into words
words = alice_text.split()

# print(trim_word(words[69]))

# Loop through each word
for word in words:
    # Trim the word
    word = trim_word(word)

    # Ignore empty words
    if word:
        # Update word_count
        word_count[word] = word_count.get(word, 0) + 1
        total_count += 1
# Count every word in alice_text. Store your count in word_count so that:
#   word_count['word'] is your count
# Store the total word count in total_count.
# Try to ignore junk words or HTML tags


# My counts are 385 Alices and 26458 words but your count may vary a little
# base on your implementation. Unsurprisingly, the most common word is "the".
print("Count of the word Alice: ", word_count["ALICE"])
print("Total word count: ", total_count)
print("Most common word: ", sorted(word_count.keys(), key=lambda w: word_count[w])[-1])

Count of the word Alice:  385
Total word count:  26458
Most common word:  THE


In [27]:
# FILL IN THIS SECTION

following_words = {}

# Count which words follow each other in alice_text. Store your counts in
# following_words so that:
#   following_words[leading word][trailing trailing] is your count
# If you implement this section correctly, following_words should be a "2D"
# dictionary

# Loop through each word
for i in range(len(words) - 1):
    # Trim the leading and trailing words
    leading_word = trim_word(words[i])
    trailing_word = trim_word(words[i + 1])

    # Ignore empty words
    if leading_word and trailing_word:
        # Update following_words
        if leading_word not in following_words:
            following_words[leading_word] = {}
        following_words[leading_word][trailing_word] = following_words[leading_word].get(trailing_word, 0) + 1

print(following_words)


# My count of how many times the word "was" came after the word "Alice" is 16
# but your count might be slightly different based on implementation
print(f"Alice is followed by was {following_words['ALICE']['WAS']} times")

{"ALICE'S": {'ADVENTURES': 1, 'FIRST': 1, 'RIGHT': 1, 'LOVE': 1, 'ELBOW': 1, 'HEAD': 1, 'AND': 1, 'SIDE': 1, 'SHOULDER': 2, 'GREAT': 1, 'EVIDENCE': 1}, 'ADVENTURES': {'IN': 1, 'I': 1, 'FIRST': 1, 'FROM': 1, 'OF': 1, 'TILL': 1}, 'IN': {'WONDERLAND': 2, 'IT': 8, 'HER': 15, 'THAT': 8, 'TIME': 4, 'ANOTHER': 7, 'THE': 79, 'A': 97, 'HAND': 2, 'SIGHT': 3, 'WAITING': 1, 'LARGE': 1, 'FACT': 4, 'MY': 9, 'CRYING': 1, 'CURRANTS': 1, 'THIS': 4, 'ONE': 3, 'SUCH': 4, 'RINGLETS': 1, 'WITH': 3, 'EXISTENCE': 1, 'SALT': 1, 'LIKE': 1, 'TRYING': 1, 'AN': 10, 'WHICH': 4, 'SILENCE': 3, 'DESPAIR': 1, 'YOUR': 2, 'REPLY': 5, 'CHORUS': 1, 'PARTICULAR': 2, 'BED': 1, 'WITHOUT': 1, 'GREAT': 2, 'HERE': 2, 'AT': 5, "BILL'S": 1, 'SPITE': 1, 'ITS': 2, 'ANY': 1, 'CURVING': 1, 'AMONG': 1, 'SAYING': 1, 'BRINGING': 1, 'LIVERY': 3, 'KNOCKING': 1, 'THERE': 1, 'SHE': 2, 'ASKED': 1, 'TALKING': 1, 'ALL': 5, 'SOME': 2, 'MARCH': 1, 'FRONT': 2, 'HIS': 5, 'AS': 1, 'ASKING': 1, 'QUESTIONS': 1, 'BY': 1, 'COUPLES': 1, 'MANAGING': 1, '

In [28]:
# FILL IN THIS SECTION

def p_that_b_follows_a(a: str, b: str) -> float:
    """
    The function returns the probability that word b follows word a.
    """
    # Get the total occurrences of word a
    total_a_occurrences = sum(following_words[a].values())

    # Get how many times b follows a
    b_after_a_count = following_words[a][b]

    # Calculate and return the probability
    if total_a_occurrences == 0:
        return 0.0  # Avoid division by zero
    return b_after_a_count / total_a_occurrences

In [29]:
# DO NOT CHANGE

# This loop will show you the likelihood of every word following 'Alice' in the
# text. I have calculated that 'did' follows Alice 2.9% of the time and 'to' follows
# Alice about 2.3% of the time. Your values should be similar.

for word in following_words['ALICE']:
  print(f"The likelihood of {word} following Alice is: {p_that_b_follows_a('ALICE', word)}")

The likelihood of WAS following Alice is: 0.04415584415584416
The likelihood of WITHOUT following Alice is: 0.0025974025974025974
The likelihood of THINK following Alice is: 0.0025974025974025974
The likelihood of STARTED following Alice is: 0.0025974025974025974
The likelihood of AFTER following Alice is: 0.0025974025974025974
The likelihood of HAD following Alice is: 0.02857142857142857
The likelihood of TO following Alice is: 0.023376623376623377
The likelihood of SOON following Alice is: 0.005194805194805195
The likelihood of BEGAN following Alice is: 0.01818181818181818
The likelihood of LIKE following Alice is: 0.0025974025974025974
The likelihood of OPENED following Alice is: 0.0025974025974025974
The likelihood of IT following Alice is: 0.012987012987012988
The likelihood of AND following Alice is: 0.04935064935064935
The likelihood of VENTURED following Alice is: 0.01038961038961039
The likelihood of I following Alice is: 0.02077922077922078
The likelihood of WHEN following Al

In [30]:
# FILL IN THIS SECTION

def get_random_starting_word() -> str:
    """
    This function should return a random word from alice_text with a probability
    given by how often the word shows up in the text. So "The" (because it appears
    1603 times) should be frequently returned by this funciton, but "Lewis" should
    be less common (because it appears 1 time in the text).
    """
    # Finding starting words of sentences in the text
    starting_words = []
    sentences = re.split(r'[.!?]\s*', alice_text)  # Split text into sentences based on punctuation

    # Extract the first word from each sentence
    for sentence in sentences:
        first_word = sentence.split()[0] if sentence.split() else None
        if first_word:
            cleaned_word = trim_word(first_word)
            if cleaned_word:  # Check if the trimmed word is non-empty
                starting_words.append(cleaned_word)

    # Count frequencies of each starting word
    freq = {}
    for word in starting_words:
        if word in freq:
            freq[word] += 1
        else:
            freq[word] = 1

    # Prepare a list with words repeated by their frequencies to sample from them
    weighted_words = []
    for word, count in freq.items():
        weighted_words.extend([word] * count)

    # Randomly select a starting word based on the frequencies
    return random.choice(weighted_words) if weighted_words else None

In [31]:
# FILL IN THIS SECTION

def get_random_next_word(start_word: str) -> str:
    """
    This function should return a random word from alice_text with a probability
    given by how often the word shows follows start_word. So if start_word is
    "Alice", your function should be very likely to return "was", but not very likely
    to return "folded".
    """
    if not start_word in following_words:
        return None  # If the start word has no recorded following words

    next_words = following_words[start_word]
    total_occurrences = sum(next_words.values())

    # Create a weighted list of next words based on their occurrence frequency
    weighted_next_words = []
    for word, count in next_words.items():
        weighted_next_words.extend([word] * count)

    # Randomly select a next word from the weighted list
    return random.choice(weighted_next_words) if weighted_next_words else None

In [34]:
# DO NOT CHANGE

# This block will print a random string of English-sounding words using your
# hidden markov model. If your code is working correctly you should see a string
# of 16 words that mimic the style of Alic in Wonderlan.

word = get_random_starting_word()
print(word, end=' ')
for _ in range(15):
  word = get_random_next_word(word)
  print(word, end=' ')
print('.')

I SEE MISS ALICE RATHER SLEEPY AND BROKE TO BE GRAND CERTAINLY BUT OUT-OF-THE-WAY THINGS SAID .
